In [ ]:
%pip install llama-index-agent-openai
%pip install llama-index

In [54]:
import os

os .environ["OPENAI_API_KEY"] = ""
from llama_index.agent.openai import OpenAIAssistantAgent
cfo_agent = OpenAIAssistantAgent.from_new(
    name="CFO Agent",
    instructions="You are a personal financial assistant, here to provide insights and analysis on company financials. Always use the documnets provided to curate answers, dont rely on previous knowledge.",
    openai_tools=[{"type": "retrieval"}],
    instructions_prefix="Please address me as CFO.",
    files=["./financial_stament_review.pdf"],
    verbose=True,
)

In [16]:
response = cfo_agent.chat(
    "what is revenue for the year 2008?"
)
print(str(response))

The revenue for the year ending December 31, 2008, for Cadillac Ranch, Incorporated was $232,000 . If you need further information or have more questions, feel free to ask!


In [52]:
cmo_agent = OpenAIAssistantAgent.from_new(
    name="CMO",
    instructions="Hello! you are a personal brand assistant, here to provide you with  expert branding and marketing insights!. Always use the documnets provided to curate answers, dont rely on previous knowledge.",
    openai_tools=[{"type": "retrieval"}],
    instructions_prefix="Please address me as CMO.",
    files=["./Marketing.pdf"],
    verbose=True,
)


In [53]:
new_response = cmo_agent.chat("summary of marketing trends of a year, could be any")
print(new_response)

The document titled "Marketing.pdf" is a campaign planning guide published in 2017 and it provides insights into the development of successful marketing campaigns for the arts and cultural sector. Here's a summary of the key points and marketing trends reflected in the guide:

**Campaign Planning Essentials:** 
- Campaign plans are part of overall marketing strategies designed to promote specific products, performances, or services. They help organize ideas, identify target markets, and formulate effective messages.
- These plans keep marketing efforts on track and provide a series of actions and targets to measure progress. They also aid in coordinating strategies between different stakeholders, such as artists and venues.

**Case Studies:**
- Northern Ballet's "Cleopatra": The campaign successfully sold £867,000 of tickets, surpassing its £832,000 target by integrating online, print, and press advertising with partnerships for events and packages.
- Frozen Light's "The Forest": By wo

In [25]:
from llama_index.agent.openai import OpenAIAssistantAgent
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [44]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./resume"
    )
    resume_index = load_index_from_storage(storage_context)
    index_loaded = True
except:
    index_loaded = False

In [45]:
print(index_loaded)

True


In [39]:
if not index_loaded:
    # load data
    resume_data = SimpleDirectoryReader('./resume').load_data()

    # build index
    resume_index = VectorStoreIndex.from_documents(resume_data)

    # persist index
    resume_index.storage_context.persist(persist_dir="./resume")


In [46]:
engine = resume_index.as_query_engine(similarity_top_k=3)

In [47]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=engine,
        metadata=ToolMetadata(
            name="Recruiter",
            description=(
                "Recruiter, suggests best candidate for the roles need to be placed"
            ),
        ),
    )
]

In [48]:
agent = OpenAIAssistantAgent.from_new(
    name="Recruiter",
    instructions="You are a recruiter, and can tell best candidate for the roles need to be placed",
    tools=query_engine_tools,
    instructions_prefix="Please address the user as CEO.",
    verbose=True,
    run_retrieve_sleep_time=1.0,
)

In [51]:
response = agent.chat("Tell me about the resume with name jamilu that is uploaded??")
print(response)

=== Calling Function ===
Calling function: Recruiter with args: {"input":"jamilu"}
Got output: Jamilu Jibrilla is an Electrical & Electronics Engineering student from MAU who is passionate about learning new technologies, specializing in Web Development with a focus on Javascript and React. He has experience working as a Frontend Intern at HNG, where he collaborated on group projects, engaged in remote communication, and implemented designs. Currently, he is working as a Frontend web developer at Nearby Restaurants, Chopify, and on an Admin Dashboard project. His skills include HTML, CSS, Javascript, React, Tailwind CSS, Sass, and Redux.
I have reviewed the resume for Jamilu Jibrilla, who is an Electrical & Electronics Engineering student at MAU with a passion for new technologies, particularly in the area of Web Development. He has specialized knowledge in JavaScript and React.

Here are some highlighted details from his resume:

- **Educational Background**: Studying Electrical & Ele

In [58]:
cfo_feedback = cfo_agent.chat("What qualities should we look for in a Product Manager?, in less less than 3 sentence.")
cmo_feedback = cmo_agent.chat("What skills are essential for a successful Product Manager? in less less than 3 sentence.")
recruiter_candidates = agent.chat("Provide a list of candidates for the Product Manager role. The following are recommendation from the CFO and CMO, consider their suggestions when deciding the right candidate. CFO:" +str(cfo_feedback) + "CMO"+str(cmo_feedback )


selected_candidate = recruiter_candidates


print(cfo_feedback)
print(cmo_feedback)

TypeError: can only concatenate str (not "AgentChatResponse") to str

In [57]:
print(selected_candidate)

It seems that the resume details provided earlier for Jamilu Jibrilla are focused on frontend web development and do not directly correlate to a Product Manager role. If you're looking for candidates suitable for a Product Manager position, we'll need to review different resumes or profiles that showcase experience in product management, strategic planning, and leadership.

Since Jamilu's profile does not align with the Product Manager role, you may want to consider candidates with backgrounds that include:

- Experience in leading cross-functional teams to deliver products.
- Having a track record of successfully managing the full product lifecycle.
- Being adept at market research, customer needs assessment, and competitive analysis.
- Strong communication and stakeholder management skills.

Please let me know if you have a pool of candidates to consider or if there are specific qualifications or experiences you are seeking for your Product Manager role. I can then assist you in iden